##  Viết nhanh level -1

In [ ]:
import os
game_name = 'WelcomeToTheDungeon_v1'
start = ',\n  "-1": [\n   [],\n   [],\n   ['
print(start)
for i in os.listdir(f'A:\ENV\Agent\Ifelse\{game_name}'):
    print(f'    "{i[:-3]}"', end=',\n')
end = '   ]\n  ]'
print(end)

In [ ]:
import json
a = json.load(open('A:\ENV\Log\level_game.json'))
for i in a:
    print(i, '-------------', list(a[i].keys())[2:])

##  Kiểm tra hiệu suất của mô hình

In [1]:
##  Kiểm tra hiện suất của hệ thống
#  !git clone https://github.com/ngoxuanphong/ENV.git
%cd A:/ENV
import warnings 
warnings.filterwarnings('ignore')
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning,NumbaExperimentalFeatureWarning, NumbaWarning
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaExperimentalFeatureWarning)
warnings.simplefilter('ignore', category=NumbaWarning)

from numba import njit
from numba.typed import List
import numpy as np
from setup import make
import os
import pandas as pd
from time import time

A:\ENV


In [2]:
def test_game(game_name, dict_test = {}):
    env = make(game_name)
    getValidActions = env.getValidActions
    numba_main_2 = env.numba_main_2

    @njit()
    def Test(state, perData):
        validActions = getValidActions(state)
        arr_action = np.where(validActions==1)[0]
        idx = np.random.randint(0, arr_action.shape[0])
        return arr_action[idx], perData
    
    per_test = [0]
    a = time()
    win1, per_test = numba_main_2(Test, 10000, per_test, 0)
    b = time()
    win2, per_test = numba_main_2(Test, 10000, per_test, 1)
    c = time()
    print(f'Game name: {game_name} | Win rate: ')
    print(f'        Level 0: {win1}/10000 in {int(b-a)} seconds')
    print(f'        Level 1: {win2}/10000 in {int(c-b)} seconds')
    try:
        win3, per_test = numba_main_2(Test, 10000, per_test, -1)
        d = time()
        time_lv_if = int(d-c)
    except:
        win3 = False
        time_lv_if = False
    if not win3:
        print(f'        Level -1: Chưa có level này')
    else:
        print(f'        Level -1: {win3}/10000 in {int(d-c)} seconds')
    dict_test[game_name] = {'Win rate': [win1, win2, win3], 
                            'Time': [int(b-a), int(c-b), time_lv_if],
                            'Graphics': os.path.exists(f'Base/{game_name}/_render_func.py')}
    return dict_test

test_game('Fantan')

Game name: Fantan | Win rate: 
        Level 0: 2423/10000 in 27 seconds
        Level 1: 107/10000 in 50 seconds
        Level -1: Chưa có level này


{'Fantan': {'Win rate': [2423, 107, False],
  'Time': [27, 50, False],
  'Graphics': False}}

In [3]:
list_game = os.listdir('Base')
dict_test = {}
for game_name in os.listdir('Base'):
    dict_test = test_game(game_name, dict_test)
df = pd.DataFrame(dict_test).T
df.to_csv('A:/ENV/Benchmark/time_win.csv')

Game name: Catan | Win rate: 
        Level 0: 2463/10000 in 318 seconds
        Level 1: 206/10000 in 191 seconds
        Level -1: 8/10000 in 188 seconds
Game name: CatanNoExchange | Win rate: 
        Level 0: 2506/10000 in 239 seconds
        Level 1: 350/10000 in 773 seconds
        Level -1: 23/10000 in 128 seconds
Game name: Century | Win rate: 
        Level 0: 2055/10000 in 39 seconds
        Level 1: 13/10000 in 51 seconds
        Level -1: Chưa có level này
Game name: Durak | Win rate: 
        Level 0: 2484/10000 in 17 seconds
        Level 1: 491/10000 in 16 seconds
        Level -1: Chưa có level này
Game name: Exploding_Kitten | Win rate: 
        Level 0: 2021/10000 in 29 seconds
        Level 1: 1639/10000 in 25 seconds
        Level -1: Chưa có level này


KeyError: 'Exploding_Kitten'

##  Test lại agent level 2

In [ ]:
#  Test lại agent lv2

def test_game(game_name):
    env = make(game_name)
    getActionSize = env.getActionSize
    getValidActions = env.getValidActions
    getReward = env.getReward
    numba_main_2 = env.numba_main_2

    @njit
    def Test(state,per):    
        actions = getValidActions(state)
        if per[-1][0][0][0][0] == 2:
            depth_tree = int(per[14][0][0][0][0])
            actions = getValidActions(state)
            output = per[-4][0][0][depth_tree] * actions + actions
            action = np.argmax(output)
        else:
            weight = per[-11][0][0][0]
            output = actions*weight + actions
            c = np.where(output == np.max(output))[0]
            action = np.random.choice(c)
            win = getReward(state)

            per[-11][0][0] += per[-9][0][0][action]
            if win != -1:
                per[-11][0][0] = np.zeros((1, getActionSize()))
        return action,per

    def convert_to_save(per):
        lst = []
        for i in range(len(per)):
            if len(per[i]) == 0:
                lst.append(List([np.zeros((1,1,1))]))
            else:
                lst.append(per[i])
        return lst

    def convert_to_test(lst):
        per = []
        for i in range(len(lst)):
            per.append(List(lst[i]))
        return per
    
    try:
        per_test = convert_to_test(np.load(f'A:\Trash\Data\{game_name}.npy', allow_pickle=True))
        win1, per_test = numba_main_2(Test, 10000, per_test, 1)
        win2, per_test = numba_main_2(Test, 10000, per_test, -1)
        print(f'Game name: {game_name} | Win rate: \n      Level 1: {win1}/10000 | Level -1: {win2}/10000')
    except:
        print(f'Game name: {game_name} | Game này chưa có level if else')

test_game('SushiGo')
for game_name in ['Splendor', 'TLMN', 'Catan']:
    test_game(game_name)
import os
list_game = os.listdir('A:\Trash\Data')
for game_name in list_game:
    test_game(game_name[:-4])

#  Test lại agent if else

In [2]:
##  Kiểm tra hiện suất của hệ thống
#  !git clone https://github.com/ngoxuanphong/ENV.git
%cd A:/ENV
import importlib.util
import warnings 
warnings.filterwarnings('ignore')
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning,NumbaExperimentalFeatureWarning, NumbaWarning
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaExperimentalFeatureWarning)
warnings.simplefilter('ignore', category=NumbaWarning)

from numba import njit
from numba.typed import List
import numba, os, sys
import numpy as np
from setup import make
game_name = 'MachiKoro'
env = make(game_name)
getActionSize = env.getActionSize
getStateSize = env.getStateSize
getAgentSize = env.getAgentSize

getValidActions = env.getValidActions
getReward = env.getReward
numba_main_2 = env.numba_main_2

A:\ENV


In [7]:
def run_game(game_name):
    path  = f'A:\ENV\Agent\Ifelse\{game_name}'
    env = make(game_name)
    def load_module_player(player, game_name = None):
        spec = importlib.util.spec_from_file_location('Agent_player', f"Agent/ifelse/{game_name}/{player}.py")
        module = importlib.util.module_from_spec(spec)
        sys.modules[spec.name] = module
        spec.loader.exec_module(module)
        return module

    for i in os.listdir(path):
        if '.py' in i:
            path_new = path + '/' + i
            #  print(path_new)
            agent = load_module_player(i[:-3], game_name = game_name)
            win, per = env.numba_main_2(agent.Test, 10000, agent.DataAgent(), 1)
            print(f'game name: {game_name}, Agent: {i[:-3]} | Win rate: {win}/1')

#  for game_name in ['Exploding_Kitten', 'Fantan', 'GoFish', 'WelcomeToTheDungeon_v1', 'WelcomeToTheDungeon_v2', 'CatanNoExchange', ]:

for game_name in ['TicketToRide']:
    run_game(game_name) 

game name: TicketToRide, Agent: PT | Win rate: 98/1


#  Graphic


In [8]:
%cd A:/ENV

A:\ENV


In [9]:
from setup import make
from Base.MachiKoro._env import bot_lv0
env = make("Durak")

In [10]:
env.render(Agent="human", per_data=[0], level=0, max_temp_frame=100)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Internal error at <numba.core.typeinfer.CallConstraint object at 0x0000023998316DD0>.
[1m[1m[1mcan't compile getActionSize: import of module _env failed[0m
[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function getActionSize at 0x00000239A59E81F0>))[0m
[0m[1mDuring: typing of call at A:\ENV\Base/Durak/_env.py (87)
[0m
Enable logging at debug level for details.
[1m
File "Base\Durak\_env.py", line 87:[0m
[1mdef getValidActions(state):
[1m    list_action = np.zeros(getActionSize())
[0m    [1m^[0m[0m


In [1]:
%cd A:/ENV

from setup import make
from Base.TicketToRide._env import bot_lv0
env = make("TicketToRide")

env.numba_main_2(bot_lv0, 10, [0], 1)

A:\ENV
check [7 2 1]
check [7 1 0]
check [8 5 0]
check [3 5 7]
check [5 0 5]
check [5 0 2]
check [1 5 0]
check [0 0 0]
check [0 0 2]
check [-1 -1 -1]
check [-1 -1 -1]
check [-1 -1 -1]
check [-1 -1 -1]
check [-1 -1 -1]
check [-1 -1 -1]
check [-1 -1 -1]
check [-1 -1 -1]
check [-1 -1 -1]
check [-1 -1 -1]


Exception: toang

In [ ]:
env.render(Agent=bot_lv0, per_data=[0], level=0, max_temp_frame=100)

In [ ]:
import time
for i in range(10):
    a = time.time()
    env.render(Agent=bot_lv0, per_data=[0], level=1, max_temp_frame=100)
    b = time.time()
    print(int(b-a), i)
    clear_output(wait=True)

In [5]:
import numpy as np
a = np.zeros(0)
a

array([], dtype=float64)